# 3. Estadística: resumiendo un histograma

In [ ]:
import ipywidgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rc("figure", dpi=100)

## Motivación

Para ver un histograma más "lindo", necesitamos muchas más mediciones.

Podemos generar datos de (una particular) manera aleatoria en Python:

In [ ]:
datos = np.random.normal(2, 0.05, size=10_000)

datos[:5]  # Muestro los 5 primeros

Podríamos redondear los números a dos cifras, para que sea más parecido a las mediciones del cronómetro:

In [ ]:
# datos = datos.round(2)

datos[:5]

Hagamos un histograma de los datos:

In [ ]:
@ipywidgets.interact(bins=ipywidgets.FloatLogSlider(min=0, max=np.log10(datos.size), readout_format=".0f"))
def _(bins):
    plt.hist(datos, bins=int(bins), histtype="step")

Si tuviesen que resumir este histograma en pocas palabras (o números), ¿qué dirían?

Podemos resumirlo en, al menos, 2 números:

1. el **centro**
2. el **ancho**

Pero, también es importante:

3. la **forma**

Comparemos con datos generados a partir de otra distribución, es decir, cuyo histograma tenga otra forma:

In [ ]:
otros_datos = np.random.uniform(*np.percentile(datos, (3, 97)), size=datos.size)

Si comparamos los números a ojo:

In [ ]:
datos[:5], otros_datos[:5]

sería muy dificil poder diferenciarlos.

Con un gráfico de puntos, no parecen tan diferentes:

In [ ]:
fig, ax = plt.subplots(1, 2, sharey=True, figsize=(8, 3))
ax[0].plot(datos[:1000], ".")
ax[1].plot(otros_datos[:1000], ".")

aunque se empiezan a ver unas diferencias.

Pero, si hacemos un histograma:

In [ ]:
@ipywidgets.interact(bins=ipywidgets.FloatLogSlider(min=0, max=np.log10(datos.size), readout_format=".0f"))
def _(bins):
    for d in (datos, otros_datos):
        plt.hist(d, bins=int(bins), histtype="step", density=True)

No tiene tanto sentido hablar de un valor central para el histograma naranja, ¿no?

**Extra**:

- La distribución naranja se llama **distribución uniforme**.

Es el análogo continuo a tirar un dado: todos los números son igual de probables.

- La distribución azul se llama **distribución normal o gaussiana**.

En general, las mediciones que hagamos en este laboratorio van a tener una distribución con forma de campana, como la azul.

Después veremos por que es razonable esperar esto, y de donde sale la gaussiana.

## Medidas de centralidad y dispersión

Para elegir un valor para el centro y el ancho, hay diferentes medidas de [centralidad](https://en.wikipedia.org/wiki/Central_tendency#Measures) y de [dispersión](https://en.wikipedia.org/wiki/Statistical_dispersion#Measures), respectivamente.

Cada una tiene distintos criterios y propiedades.

### Medidas de centralidad

Dentro de las medidas de centralidad, las más conocidas son soluciones a la pregunta:

¿cuál es el valor que está "más cerca" de todos?

Supongamos que tenemos $N$ mediciones $x_i$:

$$\{x_1, x_2, ..., x_N\}$$

y queremos determinar *el centro* $c$.

In [ ]:
@ipywidgets.interact(c=(datos.min(), datos.max(), datos.ptp() / 20))
def _(c):
    fig, axes = plt.subplots(1, 3, sharey=True, figsize=(12, 3))

    xs = [
        np.arange(30),
        datos.size - 1 - np.arange(30),
    ]
    for ax, x in zip(axes, xs):
        y = datos[x]
        ax.errorbar(x, y, yerr=(y - c, np.zeros_like(y)), fmt="o")
        ax.set(xlabel="Número de medición")

    axes[-1].hist(datos, bins="sqrt", histtype="step", orientation="horizontal")
    axes[-1].set(xlabel="Cantidad de mediciones")

    for ax in axes:
        ax.axhline(c, color="orange")

    axes[0].set(ylabel="Tiempo [s]")


¿Cómo elegimos "el mejor $c$"?

Estos métodos, empiezan calculando la distancia $d_i$ de cada punto $x_i$ al (aún desconocido) centro $c$:

$$ d_i = |x_i - c| $$

Luego, buscan un $c$ que haga todas las distancias lo más pequeñas posibles.

#### Opción 1: mediana

(*Nota: no es la que vamos a usar en este Laboratorio*)

Una opción sería elegir $c$ tal que minimize la suma de las distancias:

$$\begin{align}
S(c) &= \sum_i d_i &&= d_1 + \ldots + d_N
\\   &= \sum_i |x_i - c| &&= |x_1 -c| + \ldots + |x_N-c|
\end{align}$$

Noten que la suma es una función de una única variable, $c$, ya que los $x_i$ son valores fijos dados por nuestras mediciones.

¿Cómo encontramos el mínimo de una función de una variable?

En general, encontramos cuando la derivada es igual a $0$.

Pero esta función no es derivable. Igualmente, se puede escribir una "receta" para encontrar su mínimo.

Al $c_{min}$ que hace mínima esta suma se lo conoce como la [mediana](https://en.wikipedia.org/wiki/Median) de los $x_i$.

Se puede calcular ordenando de menor a mayor los $x_i$ y eligiendo el del medio.

Por ejemplo:

$$\begin{align}
&\text{Mediana}\Big(\{1,\,2.5,\,4,\,8,\,12\}\Big) &&= 4
\\ &\text{Mediana}\Big(\{1,\,2.5,\,4,\,8,\,12000\}\Big) &&= 4
\end{align}$$

A diferencia de la siguiente opción, la mediana es *robusta* frente a *outliers* (mediciones atípicas).

#### Opción 2: media o promedio

En lugar de la suma de las distancias, otra opción es minimizar la suma de los cuadrados de las distancias.

Este criterio, que se conoce como [cuadrados mínimos](https://en.wikipedia.org/wiki/Least_squares), es una elección estándar en estadística.

$$\begin{align}
S(c) &= \sum_i d_i^2 &&= d_1^2 + \ldots + d_N^2
\\   &= \sum_i |x_i - c|^2 &&= |x_1 - c|^2 + \ldots + |x_N - c|^2
\end{align}$$

Al igual que antes, $S(c)$ es función de una única variable.

En particular, es una parábola en $c$, y es derivable.

Para encontrar el mínimo de esta función, derivamos e igualamos a 0:

$$\begin{align*}
0 = \frac{dS}{dc} &= \frac{d}{dc} \sum_{i=1}^N (x_i - c)^2
\\&= \sum_{i=1}^N \frac{d}{dc} (x_i - c)^2
\\&= \sum_{i=1}^N -2 \, (x_i - c)
\\&= -2 \, \left( \sum_{i=1}^N x_i - \sum_{i=1}^N c \right)
\\&= -2 \, \left( \sum_{i=1}^N x_i - N c \right)
\end{align*}$$

Despejando $c$, llegamos a:

$$ c = \frac{1}{N} \sum_i^N x_i $$

Es decir, el valor óptimo (en el sentido de cuadrados mínimos) es el promedio de los $x_i$.

Generalmente se denota como $\bar{x}$ o $\langle x \rangle$.

En Python, pueden calcularla con la función `np.mean`:

In [ ]:
np.mean(datos)

**Preguntas:**

1. ¿este valor debería tener un error?

2. ¿el error depende de la cantidad de mediciones que promediamos?

### Error del promedio

Para motivar que el promedio tiene un error, vamos a gráficar el promedio para subconjuntos de distntos tamaños de las mediciones:

In [ ]:
@ipywidgets.interact(N=ipywidgets.FloatLogSlider(min=1, max=np.log10(3000), readout_format=".0f"))
def _(N):
    N = int(N)
    fig, axes = plt.subplots(3, sharex=True, gridspec_kw=dict(hspace=0))
    fig.suptitle(f"{N} datos")
    for i, ax in enumerate(axes):
        color = f"C{i}"
        
        desde = 3000 * i
        hasta = desde + N
        x = datos[desde:hasta]
        
        ax.hist(x, bins="sqrt", histtype="step", lw=2, color=color, label=f"{desde}-{hasta}")
        ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))
        
        # ax.axvline(np.mean(x), linestyle="--", lw=3, color=color)

        ax.set(yticks=())
    ax.set_xlim(datos.min(), datos.max())

Al ir aumentando el número de mediciones, los promedios coinciden cada vez más entre sí.

Es razonable persar que un promedio calculado con más mediciones va a tener menos error.

¿Cómo lo calculamos?

El promedio es:

$$ \bar{x} = \frac{1}{N} \sum_i x_i = \frac{x_1 + \ldots + x_N}{N} $$

que es una cuenta a partir nuestras mediciones $x_i$.

Es decir, el promedio $\bar{x}$ es una función de las variables $x_1, \ldots, x_N$:

$$\bar{x} = \bar{x}(x_1, \ldots, x_N)$$

Propagando, podemos calcular el error del promedio:

$$ \begin{align*}
\Delta\bar{x}^2
&= \left(\frac{\partial \bar{x}}{\partial x_1} \Delta x_1\right)^2
+ \ldots
+ \left(\frac{\partial \bar{x}}{\partial x_N} \Delta x_N\right)^2
\\ \\
&= \left(\frac{1}{N} |x_1 - \bar{x}|\right)^2
+ \ldots
+ \left(\frac{1}{N} |x_N - \bar{x}|\right)^2
\\ \\
&= \frac{1}{N^2} \Big[(x_1 - \bar{x})^2
+ \ldots
+ (x_N - \bar{x})^2\Big]
\end{align*} $$

Si pasamos la raíz:

$$
\Delta\bar{x}
= \frac{1}{N} \sqrt{(x_1 - \bar{x})^2
+ \ldots
+ (x_N - \bar{x})^2}
$$

(más adelante, simplificaremos aún más esta formula)

**Pregunta:** ¿Este valor disminuye con la cantidad de mediciones $N$? ¿De qué manera?

#### Mediciones independientes

No lo mencionamos antes, pero la fórmula de propagación asume que las mediciones son independientes.

Si no lo fueran, hay que agregar unos términos extra a la fórmula de propagación (que no vamos a ver).

- ¿Qué serían mediciones independientes?

Por ejemplo, las mediciones que hicimos del periodo entre chasquidos: el error que tuvieron, no depende de lo que midió el resto.

- ¿Cómo serían mediciones no independientes?

Por ejemplo, dos periodos consecutivos entre chasquidos.

In [ ]:
@ipywidgets.interact(m=(0.7, 1.3, 0.02))
def _(m=1):
    plt.figure(figsize=(6, 2))
    x = np.array([0, m, 2])
    plt.errorbar(np.arange(3), np.ones(3), yerr=0.4, capsize=10, label="Real")
    plt.errorbar(x, np.zeros(3), yerr=0.4, capsize=10, label="Medición")

    plt.text(0.1, 0.1, f"$\\tau_1$ = {m:.2f}")
    plt.text(1.6, 0.1, f"$\\tau_2$ = {2-m:.2f}")

    plt.xticks(
        [0, 1, 2, 0.0001, m+0.0001, 2.0001],
        [0, 1, 2, *[f"\n$t_{i}$" for i in (0, 1, 2)]]
    )
    plt.yticks([])
    plt.xlabel("Tiempo [s]")
    plt.legend(bbox_to_anchor=(1, 1))

El primer periodo $\tau_1$ es:

$$ \tau_1 = t_1 - t_0 $$

donde $t_0$ y $t_1$ es lo que marcaba el reloj al medir.

El segundo periodo $\tau_2$ es:

$$ \tau_2 = t_2 - t_1 $$

Tanto $\tau_1$ como $\tau_2$ usan la misma medición $t_1$. ¡No son independientes!

### Checkpoint

Recapitulemos, por si se perdieron en la matemática:

- Medimos (o simulamos medir)
- Visualizamos
    - El histograma tiene forma de campana.
- Resumimos con un intervalo
    - Calculamos un centro: el promedio $\bar{x}$ y su error $\Delta \bar{x}$
    - Nos falta calcular un ancho

### Medidas de dispersión

Las medidas de dispersión cuantifican que tanto se apartan las mediciones entre sí, o de su centro.

In [ ]:
@ipywidgets.interact(c=(datos.min(), datos.max(), datos.ptp() / 20))
def _(c):
    fig, axes = plt.subplots(1, 3, sharey=True, figsize=(12, 3))

    xs = [
        np.arange(30),
        datos.size - 1 - np.arange(30),
    ]
    for ax, x in zip(axes, xs):
        y = datos[x]
        ax.errorbar(x, y, yerr=(y - c, np.zeros_like(y)), fmt="o")
        ax.set(xlabel="Número de medición")

    axes[-1].hist(datos, bins="sqrt", histtype="step", orientation="horizontal")
    axes[-1].set(xlabel="Cantidad de mediciones")

    for ax in axes:
        ax.axhline(c, color="orange")

    axes[0].set(ylabel="Tiempo [s]")


#### Desviación estándar

La medida que es "consistente" con el promedio se conoce como la *desviación estándar* $s$.

La idea es hacer un promedio de las distancias $d_i$ de cada medición $x_i$ al promedio $\bar{x}$:

$$ d_i = |x_i - \bar{x}| $$

Pero, para ser "consistentes" con la forma de medir distancias del promedio, consideramos la distancias al cuadrado:

$$ d_i^2 = (x_i - \bar{x})^2 $$

Hacemos un promedio de esas distancias:

$$ \frac{1}{N} \sum_i^N (x_i - \bar{x})^2 $$

Pero si $x_i$ tiene ciertas unidades, por ejemplo, metros, eso tendría unidades de metros al cuadrado.

Entonces, tomamos la raíz de ese promedio:

$$ s = \sqrt{\frac{1}{N} \sum_i^N (x_i - \bar{x})^2} $$

*Nota: tambíen la pueden ver definida con $N-1$ en lugar de $N$, ver [la corrección de Bessel](https://en.wikipedia.org/wiki/Bessel's_correction).*

En Python, pueden calcularla con la función `np.std` (del inglés, *standard deviation*):

In [ ]:
np.std(datos)

#### Opción mala: máximo - mínimo

En general, es una malísima opción para el tipo de mediciones que vamos a hacer en el laboratorio.

Veamos que pasa al calcular el máximo y el mínimo en función de la cantidad de datos que tomamos.

In [ ]:
N = np.geomspace(1, datos.size, 100, dtype=int)

fig, ax = plt.subplots(1, 2, sharey=True, figsize=(8, 3))

ax[0].plot(datos[:100], "o")
ax[0].set(xlabel="Número de medición", ylabel="Tiempo [s]")

ax[1].plot(N, [np.max(datos[:n]) for n in N], label="Maximo")
ax[1].plot(N, [np.min(datos[:n]) for n in N], label="Minimo")
ax[1].set(xscale="log", xlabel="Cantidad de mediciones")
ax[1].legend()
for a in ax:
    a.grid()

fig, ax = plt.subplots(1, 2, figsize=(8, 3))
ax[0].hist(datos, bins="sqrt")
ax[0].set(xlabel="Tiempo [s]")
ax[1].plot(N, [np.ptp(datos[:n]) for n in N], label="Maximo-Minimo")
ax[1].plot(N, [np.std(datos[:n]) for n in N], label="Desv. est.")
ax[1].set(xscale="log", xlabel="Cantidad de mediciones", ylabel="Ancho [s]")
ax[1].legend()
ax[1].grid()

Las mediciones que están más alejadas del centro, tienen menor probabilidad de salir.

Las chances de que tengamos una dentro de nuestras mediciones, depende de cuantasm mediciones hayamos hecho.

- Con "máximo-mínimo", solo estamos considerando estos extremos para calcular el ancho.

Por lo tanto, depende de la cantidad de mediciones.

- Con la desviación estándar, estamos considerando todas las mediciones para calcular el ancho.

Por lo tanto, no es tan sensible a la cantidad de mediciones.

#### Error del promedio: parte 2

Antes, habiamos llegado a esta fórmula para el error del promedio:

$$
\Delta\bar{x}
= \frac{1}{N} \sqrt{(x_1 - \bar{x})^2
+ \ldots
+ (x_N - \bar{x})^2}
$$

Pero se puede escribir más fácilmente en términos de la desviación estándar:

$$ \Delta \bar{x} = \frac{s}{\sqrt{N}} $$

que es como lo verán en todos lados.

En Python, no hay una función que lo calcule directamente, pero podemos calcularlo así:

In [ ]:
np.std(datos) / np.size(datos)**0.5

Al final, $\Delta \bar{x}$ no decrece como $N$, sino como $\sqrt N$ (porque $s$ "no depende" de $N$).

Entonces, tomando $100$ veces más mediciones, $\Delta \bar{x}$ se reduce un factor $\sqrt{100}=10$ (es decir, ganamos un decimal).

## Interpretación estadística

Recapitulemos:

- Medimos
- Visualizamos -> Histograma con forma de campana
- Resumimos -> Intervalo

En particular, calculamos dos intervalos alrededor del promedio $\bar{x}$:

1. Con su error $\Delta \bar{x} \rightarrow \bar{x} \pm \Delta \bar{x}$
2. Con la desviación estándar $\sigma \rightarrow \bar{x} \pm s$

Pero, ¿qué representan cada uno?

In [ ]:
promedio = np.mean(datos)
desv_est = np.std(datos)
error_promedio = desv_est / np.size(datos)**0.5

fig, axes = plt.subplots(1, 2, sharey=True, figsize=(8, 2))

for ax in axes:
    ax.hist(datos, bins="sqrt", histtype="step", color="black")
    ax.axvline(promedio, label=r"$\bar{x}$", lw=3)
    ax.axvspan(promedio - desv_est, promedio + desv_est, label=r"$\bar{x} \pm s$", alpha=0.5, color="C2")
    ax.axvspan(promedio - error_promedio, promedio + error_promedio, label=r"$\bar{x} \pm \Delta \bar{x}$", alpha=1, color="C1")

axes[0].legend()
axes[1].set(xlim=(promedio - 5 * error_promedio, promedio + 5 * error_promedio))

El intervalo dado por el error del promedio, $\Delta \bar{x}$, donde

$$ \Delta \bar{x} = \frac{s}{\sqrt N} $$

nos dice que tan bien conocemos donde está el centro de nuestras mediciones.

En cambio, el intervalo dado por la desviación estándar $s$,

nos permite describir **un** rango donde caen gran parte de las mediciones:

$$ \bar{x} \pm s $$

1. ¿Por qué usamos este intervalo que no cubre todas las mediciones?

Este rango no depende (tanto) de la cantidad de mediciones:

a partir de $N \sim 10$, parece no variar mucho.

2. ¿Cuántas mediciones caen en ese intervalo?

Para calcularlo con NumPy, veamos un ejemplo más simple:

In [ ]:
x = np.array([1, 2, 10, 11])

x < 5

Cuando hacemos una comparación, nos devuelve `True` o `False` para cada elemento.

Podemos usar ese resultado para seleccionar los elementos donde se cumple la condición:

In [ ]:
x[x < 5]

y después ver cuantos elementos quedaron.

O sumarlo, ya que interpreta `True=1` y `False=0`:

In [ ]:
np.sum(x < 5)

Volviendo a la pregunta, ¿cuántas mediciones caen en el intervalo $\bar{x} \pm s$?

Es lo mismo que preguntar para cuantos $x_i$ se cumple $|x_i - \bar{x}| < s$:

In [ ]:
cond = np.abs(datos - np.mean(datos)) < np.std(datos)

np.sum(cond)

¿Y a que proporción de los datos corresponde?

In [ ]:
np.sum(cond) / np.size(datos)

Es decir, aproximadamente el 68%, o 2/3 de las mediciones.

3. Si hicieramos otra medición, ¿dónde caería?

No podemos dar un intervalo con total seguridad, pero, probablemente, en $\bar{x} \pm s$.

Con un 68% de probabilidad*.

*no siempre es 68%, después veremos de donde sale este número y cuando tiene sentido usarlo.

Entonces, podríamos decir que **$s$ es la precisión que se tiene al hacer una medición**, ¿no?

Recordemos la formula para la desviación estándar $s$,

$$ s = \sqrt{\frac{1}{N} \sum_i (x_i - \bar{x})^2} $$

4. Si la medición cae afuera de ese intervalo, ¿"es distinta"?


A diferencia de mediciones donde estamos limitados por la resolución del instrumento,

en este tipo de mediciones no es tan blanco y negro, es más gris.

Lo que se hace generalmente es decir "a cuántas desviaciones estándar" está nuestra medición del promedio.

Por ejemplo, dada una nueva medición $x_{nueva}$, y dados $\bar{x} = 5$ y $s = 2$, si:

- $x_{nueva} = 7$ está a 1 desviación estándar, porque $x_{nueva} = \bar{x} + 1 \, s$

- $x_{nueva} = 10$ está a 2.5 desviación estándar, porque $x_{nueva} = \bar{x} + 2.5 \, s$

En general, la distancia $Z$ está dada por:

$$ Z = \frac{x_{nueva} - \bar{x}}{s} $$

que se lo llama *Z-score* en estadística.

- Para $Z < 1$, "son iguales" (o, al menos, no puedo decir que sean distintos).
- Para $Z > 5$, seguro son distintos.
- En el medio, gris. No podemos asegurar que sean distintos, pero...

Este mismo criterio lo vamos a usar para comparar tanto con el intervalo dado por la desviación estándar $s$ como el del error del promedio $\bar{x}$:

1. $\bar{x} \pm s$
2. $\bar{x} \pm \Delta x$

**Pregunta**: si queremos reportar un resultado para un conjunto de mediciones, ¿cuál de estos dos reportamos?

In [ ]:
promedio = np.mean(datos)
desv_est = np.std(datos)
error_promedio = desv_est / np.size(datos)**0.5

fig, axes = plt.subplots(1, 2, sharey=True, figsize=(8, 2))

for ax in axes:
    ax.hist(datos, bins="sqrt", histtype="step", color="black")
    ax.axvline(promedio, label=r"$\bar{x}$", lw=3)
    ax.axvspan(promedio - desv_est, promedio + desv_est, label=r"$\bar{x} \pm s$", alpha=0.5, color="C2")
    ax.axvspan(promedio - error_promedio, promedio + error_promedio, label=r"$\bar{x} \pm \Delta \bar{x}$", alpha=1, color="C1")

axes[0].legend()
axes[1].set(xlim=(promedio - 5 * error_promedio, promedio + 5 * error_promedio))

Recordemos:

1. $\bar{x} \pm \Delta \bar{x}$ es que tan bien conocemos el centro,
2. $\bar{x} \pm s$ es un intervalo que contiene $2/3$ de las mediciones,

donde $s$ sería el error de cada medición.

## Interpretación física

La estadística les da herramientas para analizar los datos, pero no les dice que herramienta usar ni que concluir.

Es importante **interpretarlos en base a nuestro experimento**.

Veamos dos casos extremos, y si identificamos a cual corresponde cada intervalo.

### Caso A

En el experimento que realizamos al principio, la medición del intervalo de tiempo entre chasquidos:

¿Qué representa la desviación estándar $s$ de los datos?

¿A qué se debe *"físicamente"* la dispersión en nuestras mediciones?

Es su tiempo de reacción para accionar el cronómetro.

El tiempo entre los chasquidos tiene un valor único (desconocido, pero único).

### Caso B

Imaginemos si hubiesemos medido de otra manera muy precisa, por ejemplo, grabando el audio:

In [ ]:
from scipy.io import wavfile

rate, data = wavfile.read("chasquido.wav")
t = np.arange(data.size) / rate

fig, ax = plt.subplots(figsize=(6, 2))

ax.plot(t, data)

ax1 = ax.inset_axes([0.1, -1.2, 0.3, 0.8])
i, j = np.searchsorted(t, (0.15, 0.17))
ax1.plot(t[i:j], data[i:j])
ax.indicate_inset_zoom(ax1)

ax2 = ax.inset_axes([0.6, -1.2, 0.3, 0.8])
i, j = np.searchsorted(t, (1.16, 1.18))
ax2.plot(t[i:j], data[i:j])
ax.indicate_inset_zoom(ax2)

for a in (ax, ax1, ax2):
    a.set(xlabel="Tiempo [s]")

y definiendo como "chasquido" el instante donde es máxima la amplitud del sonido.

En comparación, la precisión para ese tiempo entre chasquidos sería altisima (menos de 1 ms).

¿Qué esperarían que pase si repetimos el experimento múltiples veces?

Es decir, grabar múltiples chasquidos.

Las mediciones van a variar, pero no por culpa del proceso de medición, sino que es una variación propia del fenómeno que queremos medir.

¿Y podrían reportar "el periodo entre chasquidos"?

No va a haber **un** peridoo entre chasquidos, ¿no?

Podrían reportar un intervalo que describa donde caigan la mayor parte de los chasquidos.

### Casos e intervalos

Entonces, considerando estos dos extremos, ¿qué reportarían en cada caso?

**Intervalos:**

1. $\bar{x} \pm \Delta \bar{x}$ es que tan bien conocemos el centro,
2. $\bar{x} \pm s$ es un intervalo que contiene $2/3$ de las mediciones,

donde $s$ sería el error de cada medición.

**Casos:**

- A. Nuestro experimento: múltiples mediciones de un único intervalo entre dos chasquidos.

- B. El experimento hipótetico del análisis de audio: múltiples mediciones de distintos intervalos entre chasquidos.

In [ ]:
promedio = np.mean(datos)
desv_est = np.std(datos)
error_promedio = desv_est / np.size(datos)**0.5

fig, axes = plt.subplots(1, 2, sharey=True, figsize=(8, 2))

for ax in axes:
    ax.hist(datos, bins="sqrt", histtype="step", color="black")
    ax.axvline(promedio, label=r"$\bar{x}$", lw=3)
    ax.axvspan(promedio - desv_est, promedio + desv_est, label=r"$\bar{x} \pm s$", alpha=0.5, color="C2")
    ax.axvspan(promedio - error_promedio, promedio + error_promedio, label=r"$\bar{x} \pm \Delta \bar{x}$", alpha=1, color="C1")

axes[0].legend()
axes[1].set(xlim=(promedio - 5 * error_promedio, promedio + 5 * error_promedio))

En la práctica, puede que se encuentren con una combinación de ambos extremos:

que la precisión del proceso de medición y la variación intrínseca del fenómeno sean similares.

Por ejemplo, múltiples mediciones del disintos intervalos entre chasquidos.

## Resumen

- Medimos
- Visualizamos -> Histograma con forma de campana
- Resumimos -> Intervalo

¿Cuál? Depende del experimento:

1. Con el error del promedio $\rightarrow \bar{x} \pm \Delta \bar{x}$
2. Con la desviación estándar $\rightarrow \bar{x} \pm s$

## Nuestro experimento

Carguemos nuestras mediciones y respondamos si el tiempo entre chasquidos fue distinto.

In [ ]:
data = pd.read_csv("mediciones.csv")

data.head()

Podemos calcular el promedio de la columna `tiempo_1`:

In [ ]:
np.mean(data.tiempo_1)

`pandas` nos deja hacerlo de forma fácil con todas las columnas:

In [ ]:
data.filter(like="tiempo").mean()

Entonces, ¿son distintos?

![Necesito calcularte el error](necesito_meme.png)

No hay una función que calcule el error directamente:

In [ ]:
np.std(data.tiempo_1) / np.size(data.tiempo_1)**0.5

Pero la pueden hacer ustedes:

In [ ]:
def error_promedio(x):
    return np.std(x) / np.size(x)**0.5

error_promedio(data.tiempo_1)

In [ ]:
data.filter(like="tiempo").apply(error_promedio)

In [ ]:
def promedio(x):
    return pd.Series({
        "promedio": np.mean(x),
        "error": error_promedio(x),
    })

data.filter(like="tiempo").apply(promedio).round(3)